In [1]:
import pandas as pd
from pulp import LpMaximize, LpProblem, lpSum, LpVariable
from ipynb.fs.full.functions import display_results

##### Prepare Data

In [2]:
players = pd.read_csv('../data/player_list.csv',
                      usecols=['Position','FPPG','Salary','Nickname'])

In [3]:
salaries = {} # Individual salaries
points = {} # Individual fantasy points per game

# Loop through positions (C, PF, PG, SF, SG)
for pos in players.Position.unique():
    # Create play points and salary dictionaries
    position = players[players.Position == pos]
    position_salaries = dict(zip(position.Nickname, position.Salary))
    position_points = dict(zip(position.Nickname, position.FPPG))
    
    salaries[pos] = position_salaries
    points[pos] = position_points

##### Generate Optimal Lineup

In [4]:
position_constraints = {'PG':2, 'SG':2, 'SF':2, 'PF':2, 'C':1}
salary_cap = 60_000

# Single lineup
for lineup in range(1):
    # Define variables
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}
    
    # Create model and construct problem
    model = LpProblem(name="lineup-optimizer", sense=LpMaximize)
    player_points = [] 
    player_salary = []

    for k, v in _vars.items():
        player_salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        player_points += lpSum([points[k][i] * _vars[k][i] for i in v])
        model += lpSum([_vars[k][i] for i in v]) == position_constraints[k]
        
    model += lpSum(player_points)
    model += lpSum(player_salary) <= salary_cap
    
    model.solve()

In [5]:
display_results(model)

O̲p̲t̲i̲m̲a̲l̲ ̲L̲i̲n̲e̲u̲p̲:̲

C_Derrick_Favors
PF_Jayson_Tatum
PF_Serge_Ibaka
PG_Kemba_Walker
PG_Kyle_Lowry
SF_Brandon_Ingram
SF_Pascal_Siakam
SG_Fred_VanVleet
SG_Jrue_Holiday

Total Points: 324.45
